In [ ]:
# Install the Together API package
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 100.1 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [ ]:
import os
import pandas as pd
import together
import logging
import time
import pickle

In [ ]:
os.environ["TOGETHER_API_KEY"] = "4bd67bf8d915ebea6c3d8a07bc5d4eac77fdc9846e2068940980fdb5eff5d0ac"
print("TOGETHER_API_KEY:", os.getenv("TOGETHER_API_KEY"))

TOGETHER_API_KEY: 4bd67bf8d915ebea6c3d8a07bc5d4eac77fdc9846e2068940980fdb5eff5d0ac


In [ ]:
class TogetherLLM:
    """Together large language models with retry mechanism."""
    def __init__(self, model: str, api_key: str, temperature: float = 0.7, max_tokens: int = 120000, retries: int = 3, backoff_factor: float = 1.5):
        self.model = model
        self.api_key = api_key
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.retries = retries
        self.backoff_factor = backoff_factor
        together.api_key = api_key  # Set the API key directly

    def call(self, prompt: str) -> str:
        """Call to Together endpoint with retry mechanism."""
        attempt = 0
        while attempt < self.retries:
            try:
                response = together.Completion.create(
                    model=self.model,
                    prompt=prompt,
                    max_tokens=self.max_tokens,
                    temperature=self.temperature
                )
                # Return response if successful
                return response.choices[0].text.strip()
            except (together.error.APIError, together.error.APIConnectionError, together.error.InvalidRequestError) as e:
                logging.warning(f"Attempt {attempt + 1} failed: {e}")
                attempt += 1
                time.sleep(self.backoff_factor ** attempt)  # Exponential backoff

        # If all retries fail, raise an exception
        raise Exception(f"Failed to complete after {self.retries} retries")


In [ ]:
def load_pickle_file(file_path: str):
    """Load the pickle file into a DataFrame."""
    with open(file_path, 'rb') as file:
        return pickle.load(file)

In [ ]:
def process_chunk_sequentially(chunk: pd.DataFrame, model: TogetherLLM, previous_summary: str = "") -> str:
    """Process a chunk sequentially and return the biography section."""
    # Ensure that all values are strings
    chunk['Transkript'] = chunk['Transkript'].astype(str)

    # Combine all rows of the chunk into a single text
    chunk_text = " ".join(chunk['Transkript'].tolist())

    # Step 1: Create a prompt that considers whether there is a previous summary
    if previous_summary:
        prompt = (
            "Hier ist die bisher erstellte Biografie:\n{previous_summary}\n\n"
            "Bitte fassen Sie die folgenden Interviewinformationen zusammen und konzentrieren Sie sich auf wichtige Lebensereignisse, "
            "insbesondere auf die erwähnten Jahre und biografisch bedeutsame Ereignisse. Vermeiden Sie Wiederholungen und irrelevante Details. "
            "Integrieren Sie neue relevante Informationen in die bestehende Biografie.\n\n{chunk_text}"
        )
    else:
        prompt = (
            "Erstellen Sie eine vollständige und detaillierte Biografie basierend auf den folgenden Interviewinformationen. "
            "Konzentrieren Sie sich dabei auf wichtige Lebensereignisse und die in den Interviews erwähnten Jahre. "
            "Vermeiden Sie Wiederholungen und irrelevante Details.\n\n{chunk_text}"
        )

    # Print the prompt that will be sent to the model
    print("\n==== Nächste Eingabeaufforderung an das Modell ====\n")
    print(prompt.format(previous_summary=previous_summary, chunk_text=chunk_text))

    # Call the model with the constructed prompt
    biography_section = model.call(prompt.format(previous_summary=previous_summary, chunk_text=chunk_text))

    # Print the intermediate summary returned by the model
    print("\n==== Zwischenergebnis (Zusammenfassung des Abschnitts) ====\n")
    print(biography_section)

    return biography_section


In [ ]:
def main(pickle_file_path: str):
    # Load the pickle file
    df = load_pickle_file(pickle_file_path)

    # Initialize your model
    model = TogetherLLM(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        api_key=os.getenv("TOGETHER_API_KEY"),
        temperature=0.7,
        max_tokens=120000,
        retries=5,  # Increased retries for stability
        backoff_factor=1.5  # Exponential backoff factor
    )

    # Process each chunk sequentially
    chunk_size = 700  # Define chunk size
    previous_summary = ""

    for start in range(0, len(df), chunk_size):
        chunk = df.iloc[start:start + chunk_size]
        try:
            # Process each chunk sequentially
            previous_summary = process_chunk_sequentially(chunk, model, previous_summary)
        except Exception as e:
            logging.error(f"Fehler bei der Verarbeitung des Abschnitts: {e}")

    # Finalize the complete biography
    final_prompt = (
        "Hier ist die vollständige Biografie, die aus verschiedenen Abschnitten des Interviews extrahiert und kombiniert wurde. "
        "Bitte verfeinern Sie diese Biografie, indem Sie sie flüssiger und kohärenter machen, Redundanzen entfernen und sicherstellen, "
        "dass die Erzählung chronologisch und vollständig ist.\n\n{final_biography}"
    )

    # Print the final prompt
    final_prompt.format(final_biography=previous_summary)

    final_biography = model.call(final_prompt.format(final_biography=previous_summary))

    # Print the final biography
    print(final_biography)


pickle_file_path = '/content/transcript.pkl'
main(pickle_file_path)

<ipython-input-5-e002190390f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Transkript'] = chunk['Transkript'].astype(str)
<ipython-input-3-fce8cb8cd021>:17: DeprecationWarning: Call to deprecated function create.
  response = together.Completion.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:68: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)



==== Nächste Eingabeaufforderung an das Modell ====

Erstellen Sie eine vollständige und detaillierte Biografie basierend auf den folgenden Interviewinformationen. Konzentrieren Sie sich dabei auf wichtige Lebensereignisse und die in den Interviews erwähnten Jahre. Vermeiden Sie Wiederholungen und irrelevante Details.

[Beginn Band 1] Die Reihenfolge, in der Sie erzählen möchten, von sich aus ist eigentlich gleich, aber sinnvoll ist es oft, wenn man bei der Kindheit anfängt. Und wenn Sie sich so an ihre Kindheit zurückerinnern, was fällt Ihnen da alles ein, so spontan? Also als Jugendjahre noch? Bitte? Als Kind, die Jahre noch? Tja … Ich war (ein großer Haudegen ?) … Früher konnte man ja überall jede Ecke bald Fußballspielen, was wir eben heute nicht mehr können. Wir hatten (gewohnt ?) in Essen Westen, Holsterhausen, da hatten wir noch vier freie Plätze … schön. [Stühlerücken] Meistens war das so früher, irgendein Kind in der Familie musste ein Musikinstrument spielen oder so, also ic

ERROR:root:Fehler bei der Verarbeitung des Abschnitts: 1 validation error for TogetherErrorResponse
code
  Input should be a valid string [type=string_type, input_value=400, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
<ipython-input-5-e002190390f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Transkript'] = chunk['Transkript'].astype(str)
<ipython-input-3-fce8cb8cd021>:17: DeprecationWarning: Call to deprecated function create.
  response = together.Completion.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:68: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment va


==== Nächste Eingabeaufforderung an das Modell ====

Erstellen Sie eine vollständige und detaillierte Biografie basierend auf den folgenden Interviewinformationen. Konzentrieren Sie sich dabei auf wichtige Lebensereignisse und die in den Interviews erwähnten Jahre. Vermeiden Sie Wiederholungen und irrelevante Details.

Nein, nur Mitglied. Mitglied, ja. Ja. Ja, und für den Kindheitsbereich … wenn Sie was vergessen haben zum Kindheitsbereich … fällt Ihnen noch was ein? … Ne, vielleicht können wir ja dann noch was … Wann haben Sie denn geheiratet? ´´43. So spät, ‘43? Ja. Da waren sie schon …? Naja, 28. 28, ja, mm. Sie sind also bis 1939 in Essen gewesen … Ja. … wie ich sehe, und dann, durch den Krieg … ‘45 kam ich wieder. Mm. Die ganzen sechs Jahre also waren sie Kriegsberichtserstatter oder … Ja, beim Deutschen Nachrichtenbüro. Ja, aber nicht im Range eines Soldaten? Nein, bei M… äh … allein unterwegs. Ja … mm. Und dann in verschiedenen … Können Sie uns verraten, da haben Sie vorhin ein

ERROR:root:Fehler bei der Verarbeitung des Abschnitts: 1 validation error for TogetherErrorResponse
code
  Input should be a valid string [type=string_type, input_value=400, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
<ipython-input-5-e002190390f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Transkript'] = chunk['Transkript'].astype(str)
<ipython-input-3-fce8cb8cd021>:17: DeprecationWarning: Call to deprecated function create.
  response = together.Completion.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:68: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment va


==== Nächste Eingabeaufforderung an das Modell ====

Erstellen Sie eine vollständige und detaillierte Biografie basierend auf den folgenden Interviewinformationen. Konzentrieren Sie sich dabei auf wichtige Lebensereignisse und die in den Interviews erwähnten Jahre. Vermeiden Sie Wiederholungen und irrelevante Details.

nicht, denn … äh … (???) in der Nachkriegszeit auch mal auf der Hütte waren, da kam auch ein Zöllner, guckte bei uns in den Kochpott rein, ich sag: „Was machen Sie denn da?“. (???) Ja klar. Ja, da war auch so einer, der auch früher mein Kollege, der musste auch so ein Schwarzhändler sein. Der musste da so ne ganze Zahnarztpraxis ausgeräumt haben [lacht] (???) verhökert [lacht] … Aha, mm. Und die haben dann ganz gut gelebt, in dieser Zeit? Die Schwarzhändler ja, und die am Pütt waren, nicht? Die meisten Leute die haben sich dann auch zur, zur Zeche gedrängt, weil sie dort erst ihre Zuteilung auch kriegten, der Care-Pakete … Mm. Äh,  Scheine … die standen sich am besten m

ERROR:root:Fehler bei der Verarbeitung des Abschnitts: 1 validation error for TogetherErrorResponse
code
  Input should be a valid string [type=string_type, input_value=400, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
<ipython-input-5-e002190390f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Transkript'] = chunk['Transkript'].astype(str)
<ipython-input-3-fce8cb8cd021>:17: DeprecationWarning: Call to deprecated function create.
  response = together.Completion.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:68: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment va


==== Nächste Eingabeaufforderung an das Modell ====

Erstellen Sie eine vollständige und detaillierte Biografie basierend auf den folgenden Interviewinformationen. Konzentrieren Sie sich dabei auf wichtige Lebensereignisse und die in den Interviews erwähnten Jahre. Vermeiden Sie Wiederholungen und irrelevante Details.

Der hat ja jetzt, nach zweieinhalb Jahren Schule hat er jetzt … wird er jetzt hier in Altenessen eingesetzt. Ach so. Heute fährt er wieder Zivilstreife. Ah ja, im Wagen, im normalen, ja, Wagen … bei dem Wetter ist das ja kein Vergnügen, nicht? Unser Junge zieht sich dick an, er hat von der Wehrmacht [sic!] noch seine Knobelbecher, und Parka … Ich mein immer so, ob das jetzt Adenauer ist oder jemand anders, wenn einer schon an die 90 geht, der hat im St… der hat als Kanzler nichts mehr da zu suchen. Wir gehen mit 60 Jahren, oder 64 oder 65 Jahren, sagen wir, kommt gewiss nicht mehr … fähig, deine Arbeit zu machen, er kann ja nicht mehr fähig sein, solch einen Staat zu fü

ERROR:root:Fehler bei der Verarbeitung des Abschnitts: 1 validation error for TogetherErrorResponse
code
  Input should be a valid string [type=string_type, input_value=400, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


## Beschreibung der Biografie
Die Biografie eines Menschen, der eine bemerkenswerte Reise von einfachen Anfängen zu einem Leben der Selbstfindung und des Erfolgs gemacht hat. Sie umfasst Kindheit, Schulzeit, Karriere und persönliche Errungenschaften.

## Biografie

Ich wurde in [Ort] geboren und wuchs in einer liebevollen Familie auf. Meine Eltern arbeiteten hart, um mir und meinen Geschwistern eine gute Ausbildung und ein sicheres Zuhause zu bieten. Als Kind war ich sehr neugierig und hatte immer viele Fragen. Ich liebte es, Bücher zu lesen und mich mit Themen zu beschäftigen, die mich interessierten. Meine Eltern ermutigten mich, meine Leidenschaft für das Lernen zu verfolgen, und halfen mir, meine Ziele zu erreichen.

Mit 12 Jahren begann ich, mich für Wissenschaft und Technologie zu interessieren. Ich war fasziniert von der Möglichkeit, Probleme zu lösen und neue Dinge zu entdecken. Meine Leidenschaft für diese Themen führte mich dazu, an Wettbewerben teilzunehmen und Projekte zu e